<a href="https://colab.research.google.com/github/rflameiro/projects/blob/main/Bayes_Error_Rate_Estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import dataset

Dataset: BBBP from MoleculeNet after removing compounds that failed standardization and conversion to Morgan fingerprints (1024 bits, radius=3).

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# import dataset
url = 'https://raw.githubusercontent.com/rflameiro/Python_e_Quiminformatica/main/datasets/BBBP_morganFP_1024_radius3.csv'
df_fp = pd.read_csv(url, sep=";", index_col=False)

In [3]:
X = df_fp.iloc[:, :-1]
y = df_fp.iloc[:, -1]

print(X.shape, y.shape)

(1934, 1024) (1934,)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [5]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

# Learning to Benchmark

Adapted from [GitHub](https://github.com/mrtnoshad/Bayes_Error_Estimator/blob/master/Example.ipynb)

In [6]:
!git clone https://github.com/mrtnoshad/Bayes_Error_Estimator/

fatal: destination path 'Bayes_Error_Estimator' already exists and is not an empty directory.


Important: I found two problems using this code on Colab:

First, I had to delete the line `matplotlib.use('Agg')` on BER_estimator_KDtree.py to prevent `ImportError: Cannot load backend 'TkAgg' which requires the 'tk' interactive framework, as 'headless' is currently running`

Also, on the same file, I had to to replace `asscalar` to `ndarray.item` as it is a deprecated function.

Then I deleted the file on the Bayes_Error_Estimator folder and re-uploaded the fixed version. I already created a pull request and an issue on GitHub.

In [7]:
import sys

sys.path.append("/content/Bayes_Error_Estimator")

In [8]:
import numpy as np

from BER_estimator_KDtree import ensemble_bg_estimator as BER

In [9]:
# Estimate Bayes Error Rate
est_BER = BER(X_train, y_train)

print(est_BER)

0.02134041599271922


This indicates that for the BBBP dataset the Bayes error rate is around 2%, which seems a bit low for a cheminformatics dataset. Let's use another method and compare.

# FeeBee

These functions are used to estimate the lower and upper bounds of the BER for binary classifiers instead of just one value. They were adapted from https://github.com/DS3Lab/feebee/tree/main.

`knn_eval_from_matrices_split()` was adapted from https://github.com/DS3Lab/feebee/blob/main/methods/knn.py#L145. It uses the k-NN method and the dataset needs to be split into train and test sets. I added the option to use Tanimoto distances, as this is widely used in cheminformatics.

`ghp_eval_from_matrix()` was adapted from https://github.com/DS3Lab/feebee/blob/main/methods/ghp.py#L15. It uses the GHP method and only the train set is used.

In [10]:
!git clone https://github.com/DS3Lab/feebee

Cloning into 'feebee'...
remote: Enumerating objects: 311, done.
remote: Total 311 (delta 0), reused 0 (delta 0), pack-reused 311
Receiving objects: 100% (311/311), 812.50 KiB | 8.46 MiB/s, done.
Resolving deltas: 100% (165/165), done.


In [11]:
import sys

sys.path.append("/content/feebee")

In [12]:
import math
import numpy as np
import os.path as path
import pandas as pd
import random
from scipy.sparse.csgraph import minimum_spanning_tree
from sklearn.model_selection import train_test_split
import time

from methods.utils import compute_distance_matrix_loo, knn_errorrate, knn_errorrate_loo, load_data, load_embedding_fn
# use this instead of:
# from methods.utils import *
# because I reimplemented compute_distance_matrix

import transformations.label_noise as label_noise

## Function to calculate distances for kNN

In [13]:
# from sklearn.metrics import jaccard_score

def compute_distance_matrix(x_train, x_test, measure="tanimoto"):
    """Calculates the distance matrix between test and train.

    Args:
    x_train: Matrix (NxD) where each row represents a training sample
    x_test: Matrix (MxD) where each row represents a test sample
    measure: Distance measure (not necessarly metric) to use

    Raises:
    NotImplementedError: When the measure is not implemented

    Returns:
    Matrix (MxN) where element i,j is the distance between
    x_test_i and x_train_j.

    ------

    Removed the option to use tensorflow
    Added the option to use Tanimoto distances: measure="tanimoto"

    """

    # Type check
    if x_train.dtype != np.float32:
        x_train = np.float32(x_train)
    if x_test.dtype != np.float32:
        x_test = np.float32(x_test)

    # This method was taking too long, so I remade the code using matrix operations
    # if measure == "tanimoto":
        # M = x_test.shape[0]
        # N = x_train.shape[0]

        # # Initialize an empty MxN matrix for Jaccard distances
        # x_xt = np.zeros((M, N))

        # # Iterate through each pair of rows from x_test and x_train
        # for i in range(M):
        #     for j in range(N):
        #         # Calculate the Jaccard distance between the i-th row of x_test and the j-th row of x_train
        #         jaccard_distance = 1.0 - jaccard_score(x_test[i], x_train[j])

        #         # Store the Jaccard distance in the matrix
        #         x_xt[i, j] = jaccard_distance

    if measure == "tanimoto":
        # Assumes x_test and x_train are matrices in which each row is a binary vector.
        intersection = np.dot(x_test, x_train.T)
        union = np.sum(x_test, axis=1, keepdims=True) + np.sum(x_train, axis=1, keepdims=True).T - intersection
        # Compute Jaccard distance matrix. np.where is used to account for division by zero errors
        x_xt = np.where(union == 0, 0.0, 1.0 - (intersection / union))

    elif measure == "squared_l2":
        x_xt = np.matmul(x_test, np.transpose(x_train))

        x_train_2 = np.sum(np.square(x_train), axis=1)
        x_test_2 = np.sum(np.square(x_test), axis=1)

        for i in range(np.shape(x_xt)[0]):
            x_xt[i, :] = np.multiply(x_xt[i, :], -2)
            x_xt[i, :] = np.add(x_xt[i, :], x_test_2[i])
            x_xt[i, :] = np.add(x_xt[i, :], x_train_2)

    elif measure == "cosine":
        x_xt = np.matmul(x_test, np.transpose(x_train))

        x_train_2 = np.linalg.norm(x_train, axis=1)
        x_test_2 = np.linalg.norm(x_test, axis=1)

        outer = np.outer(x_test_2, x_train_2)
        x_xt = np.ones(np.shape(x_xt)) - np.divide(x_xt, outer)

    else:
        raise NotImplementedError("Method '{}' is not implemented".format(measure))

    return x_xt

## BER estimator: kNN

In [14]:
# knn.py

KEY_PATTERN = "measure={0}, k={1}"

def knn_get_lowerbound(value, k, classes):

    if value <= 1e-10:
        return 0.0

    if classes > 2 or k == 1:
        return ((classes - 1.0)/float(classes)) * (1.0 - math.sqrt(max(0.0, 1 - ((float(classes) / (classes - 1.0)) * value))))

    if k > 2:
        return value / float(1 + (1.0/math.sqrt(k)))

    return value / float(1 + math.sqrt(2.0/k))


def knn_eval_from_matrices_split(train_features, test_features, train_labels,
                                 test_labels, knn_measure='tanimoto',
                                 knn_k=[1,3,5,7,9], knn_subtest=None, knn_subtrain=None):

    total_classes = np.unique(np.concatenate((train_labels, test_labels))).size
    vals = {}
    ks = sorted(set(knn_k), reverse=True)

    if knn_subtest is not None:
        test_dividable = (test_features.shape[0] % knn_subtest) == 0
        test_iterations = (test_features.shape[0] + knn_subtest - 1 ) // knn_subtest
    else:
        test_dividable = True
        test_iterations = 1

    for i in range(test_iterations):
        if test_iterations == 1:
            current_pos = 0
            current_samples = test_features.shape[0]
        else:
            current_pos = i*knn_subtest
            current_samples = min(test_features.shape[0],(i+1)*knn_subtest) - current_pos

        # # if multiple train iterations
        # if knn_subtrain is not None:
        #     # select per split to nearest k (for max k) train point distances and indices
        #     train_iterations = (train_features.shape[0] + knn_subtrain -1) // knn_subtrain
        #     d = None
        #     y_train = None
        #     for j in range(train_iterations):
        #         start_idx = j * knn_subtrain
        #         end_idx = min(train_features.shape[0], (j+1) * knn_subtrain)

        #         start = time.time()
        #         d_j = compute_distance_matrix(train_features[start_idx:end_idx,:], test_features[current_pos:(current_pos + current_samples),:], knn_measure)
        #         end = time.time()


        #         if end_idx - start_idx < ks[0]:
        #             # do not run argpartition
        #             sub_d = d_j
        #             indices = np.tile(range(start_idx, end_idx), (sub_d.shape[0],1))
        #         else:
        #             # run argpartition and build
        #             indices = np.argpartition(d_j, ks[0] - 1, axis=1)
        #             # update sub_d and y_train_new
        #             num_rows = indices[:, :ks[0]].shape[0]
        #             num_cols = indices[:, :ks[0]].shape[1]
        #             rows = [x for x in range(num_rows) for _ in range(num_cols)]
        #             cols = indices[:, :ks[0]].reshape(-1)
        #             sub_d = d_j[rows, cols].reshape(num_rows, -1)

        #             indices = indices[:, :ks[0]] + start_idx

        #         y_train_new = indices
        #         for k in range(y_train_new.shape[0]):
        #             y_train_new[k, :] = train_labels[y_train_new[k, :]]

        #         if d is None:
        #             d = sub_d
        #         else:
        #             d = np.concatenate((d, sub_d), axis=1)

        #         if y_train is None:
        #             y_train = y_train_new
        #         else:
        #             y_train = np.concatenate((y_train, y_train_new), axis=1)

        #     # run knn on that smaller matrix

        #     start = time.time()
        #     err = knn_errorrate(d,
        #                         y_train,
        #                         test_labels[current_pos:(current_pos + current_samples)],
        #                         k=ks)
        #     end = time.time()


        # else:

        start = time.time()
        d = compute_distance_matrix(train_features, test_features[current_pos:(current_pos + current_samples),:], knn_measure)
        end = time.time()

        start = time.time()
        err = knn_errorrate(d,
                            train_labels,
                            test_labels[current_pos:(current_pos + current_samples)],
                            k=ks)
        end = time.time()


        if not test_dividable:
            err = [e * (current_samples/test_features.shape[0]) for e in err]
        else:
            err = [e * (1.0/test_iterations) for e in err]

        for idx, k in enumerate(ks):
            if k not in vals:
                vals[k] = err[idx]
            else:
                vals[k] += err[idx]

    results = {}
    for k, v in vals.items():
        results[KEY_PATTERN.format(knn_measure, k)] = [v, knn_get_lowerbound(v, k, total_classes)]

    return results


## BER estimator: GHP

In [15]:
# ghp.py

def ghp_eval_from_matrix(train_features, train_labels, ghp_approx=None):

    start = time.time()
    d = compute_distance_matrix_loo(train_features, "cosine")
    #d = np.sqrt(compute_distance_matrix_loo(train_features, "squared_l2"))

    if ghp_approx:
        indices = np.argpartition(d, ghp_approx, axis=1)[:,:ghp_approx]
        for row_i in range(d.shape[0]):
            mask = np.ones(d.shape[1],dtype=bool)
            mask[indices[row_i,:]] = False
            d[row_i, mask] = 0.0

    end = time.time()

    start = time.time()
    d = minimum_spanning_tree(d).tocoo()
    end = time.time()

    start = time.time()

    classes, classes_counts = np.unique(train_labels, return_counts=True)

    num_train_samples = train_labels.size
    num_classes = len(classes)

    mapping = {}
    idx = 0
    for c in classes:
        mapping[c] = idx
        idx += 1

    deltas = []
    for i in range(num_classes-1):
        deltas.append([0.0]*(num_classes-i-1))

    # Calculate number of dichotomous edges
    for i in range(num_train_samples - 1):
        label_1 = mapping[train_labels[d.row[i]]]
        label_2 = mapping[train_labels[d.col[i]]]
        if label_1 == label_2:
            continue
        if label_1 > label_2:
            tmp = label_1
            label_1 = label_2
            label_2 = tmp
        deltas[label_1][label_2 - label_1 - 1] += 1

    # Divide the number of dichotomous edges by 2 * num_train_samples to get estimator of deltas
    deltas = [[item / (2.0 * num_train_samples) for item in sublist] for sublist in deltas]

    # Sum up all the deltas
    delta_sum = sum([sum(sublist) for sublist in deltas])

    end = time.time()
    # logging.log(logging.DEBUG, "Estimators computed in {} seconds".format(end - start))

    upper = 2.0 * delta_sum

    lower = ((num_classes - 1.0) / float(num_classes)) * (1.0 - math.sqrt(max(0.0, 1.0 - ((2.0 * num_classes)/(num_classes - 1.0) * delta_sum))))

    return {"default": [upper, lower]}

## Estimate lower and upper bounds of BER on MoleculeNet BBBP dataset

In [16]:
# kNN cosine
df = knn_eval_from_matrices_split(X_train, X_test, y_train, y_test, knn_measure='cosine')
df

{'measure=cosine, k=9': [0.12144702842377261, 0.09108527131782947],
 'measure=cosine, k=7': [0.12144702842377261, 0.08813509404821888],
 'measure=cosine, k=5': [0.10852713178294573, 0.07499040371124602],
 'measure=cosine, k=3': [0.11627906976744186, 0.07371797630413504],
 'measure=cosine, k=1': [0.12661498708010335, 0.0679207173909534]}

In [17]:
# kNN Tanimoto
df = knn_eval_from_matrices_split(X_train, X_test, y_train, y_test, knn_measure='tanimoto')
df

{'measure=tanimoto, k=9': [0.11627906976744186, 0.0872093023255814],
 'measure=tanimoto, k=7': [0.12403100775193798, 0.09001030881520225],
 'measure=tanimoto, k=5': [0.10852713178294573, 0.07499040371124602],
 'measure=tanimoto, k=3': [0.11627906976744186, 0.07371797630413504],
 'measure=tanimoto, k=1': [0.13178294573643412, 0.07092130426717413]}

In [18]:
# GHP - uses cosine distance
df = ghp_eval_from_matrix(X_train, y_train)
df

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


{'default': [0.16095669036845509, 0.08826992481023144]}

It seems that the Bayes error rate for the BBBP dataset is between 0.07 and 0.12, which translates to an accuracy range of 88-93%. I believe this is a more reasonable estimate than the one from "Learning to Benchmark".

Let's see what we can get with a Random Forest model:

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

rf_classifier = RandomForestClassifier(random_state=42)

# cross-validation on the training set
cv_scores = cross_val_score(rf_classifier, X_train, y_train, cv=5)

# fit, predict test set
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)

# Evaluate accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print results
print("Cross-validation scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())
print("Test Set Accuracy:", test_accuracy)

Cross-validation scores: [0.88064516 0.87741935 0.86731392 0.89967638 0.87055016]
Mean CV Accuracy: 0.8791209938406931
Test Set Accuracy: 0.875968992248062


We got an accuracy estimate close to the upper bound of our BER estimate, so there might still be room for improvement on our model.